In [2]:
import os
import pandas as pd
import numpy as np
import gc
from glob import glob
from tqdm import tqdm
tqdm.pandas()

import pickle
#import pyarrow.parquet as pq
#import dask

C:\Users\_v010ch_\Anaconda3\lib\site-packages\tqdm\std.py:697: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [3]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [4]:
DATA = './data'
DATA_OWN = './data_own'
CLICKSTREAM = 'alfabattle2_abattle_clickstream'

In [37]:
%%time
data_train = pd.read_csv(os.path.join(DATA, 'alfabattle2_abattle_train_target.csv'), parse_dates=['timestamp']).sort_values(by = ['timestamp'])
data_train.reset_index(inplace = True)
data_train.drop('index', axis = 1, inplace = True)
data_train.index.name = 'ind'
data_train.head()

Wall time: 16.9 s


,session_id,client_pin,timestamp,multi_class_target
ind,,,,
0,da4ab6bb6795a4eb6458b88af7fb2367,70783113e4f4117935d9f746237fce3e,2020-01-01 00:00:00,mobile_recharge
1,f99d13f31091099672e51c1a02e4c567,989f645a00e3e8a179036cf5fd5be29d,2020-01-01 00:00:00,statement
2,69301aa932f73dec3355b803a63c39fe,586fe605f61e81581efe72d9d3dc237d,2020-01-01 00:00:01,statement
3,c6ae8307a0214b526a0e6fca3a9b80f5,c825cc9d1dfefe1fbc0fa1ead60fe2e1,2020-01-01 00:00:23,statement
4,3fd49317a617d521e0865496ec797550,29928aba9d9bbf5e0377ddc6ec6f5294,2020-01-01 00:00:35,mobile_recharge


In [38]:
data_train.shape

(5065350, 4)

form features for future using: day of month / day of week / hour / times of day

In [39]:
def get_time_of_day(inp_hour):
    if (inp_hour >= 12) and (inp_hour < 18):
        return 'day'
    elif (inp_hour >= 6) and (inp_hour < 12):
        return 'morning'
    elif (inp_hour >= 18) and (inp_hour <= 23):
        return 'evening'
    else:
        return 'night'

In [40]:
def week_moment(inp_dow):
    if (inp_dow == 5) or (inp_dow == 6):
        return 'weekend'
    else:
        return 'weekdays'

In [41]:
def holiday():
    return 'holiday'
    return 'pre-holiday'
    return 'after_the_holiday'

In [42]:
%%time
# 51.3 s

data_train['dom']  = data_train.timestamp.apply(lambda x: x.day)
data_train['dow']  = data_train.timestamp.apply(lambda x: x.weekday())
data_train['hour'] = data_train.timestamp.apply(lambda x: x.hour)
data_train['tod']  = data_train.hour.apply(get_time_of_day)
data_train['week_moment'] = data_train.dow.apply(week_moment)

Wall time: 51.3 s


calculate time difference (in hours) between sessions inside each client for future using

In [43]:
%%time
# 49.2 s

data_train['time_diff'] = data_train.groupby(['client_pin']).timestamp.diff()#.agg('size')
data_train['time_diff'] = data_train.time_diff.apply(lambda x: x.total_seconds() / 3600)
data_train['time_diff'].fillna(0, inplace = True)

Wall time: 48.8 s


In [44]:
data_train.head()

,session_id,client_pin,timestamp,multi_class_target,dom,dow,hour,tod,week_moment,time_diff
ind,,,,,,,,,,
0,da4ab6bb6795a4eb6458b88af7fb2367,70783113e4f4117935d9f746237fce3e,2020-01-01 00:00:00,mobile_recharge,1,2,0,night,weekdays,0.0
1,f99d13f31091099672e51c1a02e4c567,989f645a00e3e8a179036cf5fd5be29d,2020-01-01 00:00:00,statement,1,2,0,night,weekdays,0.0
2,69301aa932f73dec3355b803a63c39fe,586fe605f61e81581efe72d9d3dc237d,2020-01-01 00:00:01,statement,1,2,0,night,weekdays,0.0
3,c6ae8307a0214b526a0e6fca3a9b80f5,c825cc9d1dfefe1fbc0fa1ead60fe2e1,2020-01-01 00:00:23,statement,1,2,0,night,weekdays,0.0
4,3fd49317a617d521e0865496ec797550,29928aba9d9bbf5e0377ddc6ec6f5294,2020-01-01 00:00:35,mobile_recharge,1,2,0,night,weekdays,0.0


In [12]:
#data.groupby(['client_pin']).multi_class_target.agg(['nunique', 'size'])['size'].hist()

In [13]:
['last_before', 'most_common', 'mc_last_24hours', 'mc_last_week', 'days_since_last_'
 'days_since_last_mobile_recharge', 'days_since_last_statement', 'days_since_last_phone_money_transfer', 
 'days_since_last_chat', 'days_since_last_invest', 'days_since_last_main_screen', 'days_since_last_own_transfer', 
 'days_since_last_card_recharge', 'days_since_last_credit_info', 'days_since_last_card2card_transfer'
]

['last_before',
 'most_common',
 'mc_last_24hours',
 'mc_last_week',
 'days_since_last_days_since_last_mobile_recharge',
 'days_since_last_statement',
 'days_since_last_phone_money_transfer',
 'days_since_last_chat',
 'days_since_last_invest',
 'days_since_last_main_screen',
 'days_since_last_own_transfer',
 'days_since_last_card_recharge',
 'days_since_last_credit_info',
 'days_since_last_card2card_transfer']

In [14]:
#data.multi_class_target.unique()

In [15]:
def glob_freq_blank(inp_series, feature):
    tmp = inp_series.value_counts()
    return (tmp[feature]) / sum(tmp.values)

#### global frequency of target for all clients

In [16]:
def glob_freq_main_screen(inp_series):
    return glob_freq_blank(inp_series, 'main_screen')
    
def glob_freq_statement(inp_series):
    return glob_freq_blank(inp_series, 'statement')
    
def glob_freq_credit_info(inp_series):
    return glob_freq_blank(inp_series, 'credit_info')
    
def glob_freq_own_transfer(inp_series):
    return glob_freq_blank(inp_series, 'own_transfer')

def glob_freq_mobile_recharge(inp_series):
    return glob_freq_blank(inp_series, 'mobile_recharge')
          
def glob_freq_phone_money_transfer(inp_series):
    return glob_freq_blank(inp_series, 'phone_money_transfer')                             
     
def glob_freq_card2card_transfer(inp_series):
    return glob_freq_blank(inp_series, 'card2card_transfer')
       
def glob_freq_chat(inp_series):
    return glob_freq_blank(inp_series, 'chat')
                     
def glob_freq_card_recharge(inp_series):
    return glob_freq_blank(inp_series, 'card_recharge')
    
def glob_freq_invest(inp_series):
    return glob_freq_blank(inp_series, 'invest')

In [17]:
glob_for_agg = [glob_freq_main_screen,     glob_freq_statement, 
                glob_freq_credit_info,     glob_freq_own_transfer,
                glob_freq_mobile_recharge, glob_freq_phone_money_transfer, 
                glob_freq_card2card_transfer, glob_freq_chat, 
                glob_freq_card_recharge,   glob_freq_invest
               ]

In [18]:
%%time
glob_freq = data_train.multi_class_target.agg(glob_for_agg)
glob_freq.to_csv(os.path.join(DATA_OWN, 'glob_freq.csv'))
glob_freq.head()

Wall time: 4.22 s


glob_freq_main_screen        0.450268
glob_freq_statement          0.182133
glob_freq_credit_info        0.098453
glob_freq_own_transfer       0.057267
glob_freq_mobile_recharge    0.052609
Name: multi_class_target, dtype: float64

#### global frequency of target for all clients for every day of month

In [19]:
%%time 
# 5.66 s
# 5.99 s

glob_freq_dom = data_train.groupby('dom').multi_class_target.agg(glob_for_agg)
glob_freq_dom.to_csv(os.path.join(DATA_OWN, 'glob_freq_dom.csv'))
glob_freq_dom.head()

Wall time: 5.23 s


,glob_freq_main_screen,glob_freq_statement,glob_freq_credit_info,glob_freq_own_transfer,glob_freq_mobile_recharge,glob_freq_phone_money_transfer,glob_freq_card2card_transfer,glob_freq_chat,glob_freq_card_recharge,glob_freq_invest
dom,,,,,,,,,,
1,0.428351,0.195047,0.098125,0.058843,0.062329,0.044093,0.037749,0.036113,0.028268,0.011081
2,0.445593,0.180907,0.103395,0.057890,0.056484,0.043713,0.036329,0.036920,0.027012,0.011757
3,0.454650,0.176545,0.101865,0.057791,0.051958,0.045189,0.035806,0.037388,0.026956,0.011853
4,0.457900,0.173677,0.103382,0.057113,0.051556,0.044412,0.037120,0.036806,0.026383,0.011652
5,0.461488,0.175487,0.100784,0.058352,0.050210,0.044267,0.035535,0.034186,0.028329,0.011362


#### global frequency of target for all clients for every day of week

In [20]:
%%time
# 5.44 s
# 5.56 s

glob_freq_dow = data_train.groupby('dow').multi_class_target.agg(glob_for_agg)
glob_freq_dow.to_csv(os.path.join(DATA_OWN, 'glob_freq_dow.csv'))
glob_freq_dow

Wall time: 5.1 s


,glob_freq_main_screen,glob_freq_statement,glob_freq_credit_info,glob_freq_own_transfer,glob_freq_mobile_recharge,glob_freq_phone_money_transfer,glob_freq_card2card_transfer,glob_freq_chat,glob_freq_card_recharge,glob_freq_invest
dow,,,,,,,,,,
0,0.462697,0.179711,0.098724,0.055273,0.050665,0.044325,0.035949,0.035412,0.025996,0.011249
1,0.454639,0.180559,0.101255,0.055046,0.049703,0.045947,0.036778,0.037913,0.026532,0.011628
2,0.449796,0.182433,0.099672,0.056417,0.050623,0.045779,0.037156,0.038783,0.027322,0.012019
3,0.447766,0.183614,0.098479,0.056278,0.050465,0.047408,0.037882,0.038380,0.027917,0.011812
4,0.449968,0.189098,0.089751,0.058744,0.048736,0.050191,0.039099,0.035491,0.028182,0.010740
5,0.437306,0.178733,0.099890,0.061046,0.060553,0.045153,0.042636,0.035281,0.028676,0.010727
6,0.445288,0.178233,0.104193,0.059450,0.062668,0.041043,0.038895,0.032804,0.027074,0.010351


#### global frequency of target for all clients for every hour

In [21]:
%%time
# 5.73 s
# 5.52 s

glob_freq_hour = data_train.groupby('hour').multi_class_target.agg(glob_for_agg)
glob_freq_hour.to_csv(os.path.join(DATA_OWN, 'glob_freq_hour.csv'))
#glob_freq_hour.head()
glob_freq_hour.sample(5)

Wall time: 5.38 s


,glob_freq_main_screen,glob_freq_statement,glob_freq_credit_info,glob_freq_own_transfer,glob_freq_mobile_recharge,glob_freq_phone_money_transfer,glob_freq_card2card_transfer,glob_freq_chat,glob_freq_card_recharge,glob_freq_invest
hour,,,,,,,,,,
10,0.453007,0.169202,0.101545,0.057450,0.059258,0.047034,0.036493,0.037477,0.027565,0.010970
2,0.444038,0.181260,0.122340,0.044078,0.072942,0.031151,0.033587,0.035153,0.021604,0.013847
15,0.458342,0.186971,0.089368,0.060786,0.040436,0.047513,0.040703,0.037883,0.027998,0.010001
16,0.454229,0.190793,0.090207,0.060691,0.040670,0.047377,0.040766,0.036376,0.028879,0.010012
20,0.430553,0.198338,0.098321,0.059155,0.047316,0.046851,0.042054,0.036914,0.027731,0.012768


#### ???global frequency of target for all clients for every times of day????

In [22]:
%%time
# 5.7 s
# 5.65 s

glob_freq_tod = data_train.groupby('tod').multi_class_target.agg(glob_for_agg)
glob_freq_tod.to_csv(os.path.join(DATA_OWN, 'glob_freq_tod.csv'))
#glob_freq_tod.head()
glob_freq_tod

Wall time: 5.49 s


,glob_freq_main_screen,glob_freq_statement,glob_freq_credit_info,glob_freq_own_transfer,glob_freq_mobile_recharge,glob_freq_phone_money_transfer,glob_freq_card2card_transfer,glob_freq_chat,glob_freq_card_recharge,glob_freq_invest
tod,,,,,,,,,,
day,0.456649,0.185174,0.089760,0.060708,0.042505,0.049040,0.040268,0.037288,0.028409,0.010199
evening,0.431842,0.198670,0.100010,0.058700,0.046417,0.045566,0.041307,0.037583,0.027151,0.012754
morning,0.457320,0.166099,0.103246,0.054353,0.066289,0.045100,0.034454,0.035082,0.026971,0.011084
night,0.447566,0.177681,0.120468,0.045952,0.070826,0.034378,0.031950,0.034236,0.024046,0.012897


## get difference from global freq target

In [23]:
%%time
# 154 ms

glob_diff_freq_dom  = pd.DataFrame(index = glob_freq_dom.index)
glob_diff_freq_dow  = pd.DataFrame(index = glob_freq_dow.index)
glob_diff_freq_hour = pd.DataFrame(index = glob_freq_hour.index)
glob_diff_freq_tod  = pd.DataFrame(index = glob_freq_tod.index)

for el in glob_freq.keys():
    glob_diff_freq_dom['diff_' + el]   = glob_freq[el] - glob_freq_dom[el]
    glob_diff_freq_dow['diff_' + el]   = glob_freq[el] - glob_freq_dow[el]
    glob_diff_freq_hour['diff_' + el]  = glob_freq[el] - glob_freq_hour[el]
    glob_diff_freq_tod['diff_' + el]   = glob_freq[el] - glob_freq_tod[el]

    
glob_diff_freq_dom.to_csv(os.path.join(DATA_OWN, 'glob_diff_freq_dom.csv'))
glob_diff_freq_dow.to_csv(os.path.join(DATA_OWN, 'glob_diff_freq_dow.csv'))
glob_diff_freq_hour.to_csv(os.path.join(DATA_OWN, 'glob_diff_freq_hour.csv'))
glob_diff_freq_tod.to_csv(os.path.join(DATA_OWN, 'glob_diff_freq_tod.csv'))

Wall time: 329 ms


In [24]:
del glob_diff_freq_dom
del glob_diff_freq_dow
del glob_diff_freq_hour
del glob_diff_freq_tod

gc.collect()

22

In [25]:
def client_freq_blank(inp_series, feature):
    tmp = inp_series.value_counts()
    if feature in tmp.keys():
        return (tmp[feature]) / sum(tmp.values)
    else:
        return 0

#### client frequency of target for all dataset

In [26]:
def client_freq_main_screen(inp_series):
    return client_freq_blank(inp_series, 'main_screen')
    
def client_freq_statement(inp_series):
    return client_freq_blank(inp_series, 'statement')
    
def client_freq_credit_info(inp_series):
    return client_freq_blank(inp_series, 'credit_info')
    
def client_freq_own_transfer(inp_series):
    return client_freq_blank(inp_series, 'own_transfer')

def client_freq_mobile_recharge(inp_series):
    return client_freq_blank(inp_series, 'mobile_recharge')
          
def client_freq_phone_money_transfer(inp_series):
    return client_freq_blank(inp_series, 'phone_money_transfer')                             
     
def client_freq_card2card_transfer(inp_series):
    return client_freq_blank(inp_series, 'card2card_transfer')
       
def client_freq_chat(inp_series):
    return client_freq_blank(inp_series, 'chat')
                     
def client_freq_card_recharge(inp_series):
    return client_freq_blank(inp_series, 'card_recharge')
    
def client_freq_invest(inp_series):
    return client_freq_blank(inp_series, 'invest')

In [27]:
cli_for_agg = [client_freq_main_screen, client_freq_statement, 
               client_freq_credit_info, client_freq_own_transfer,
               client_freq_mobile_recharge, client_freq_phone_money_transfer, 
               client_freq_card2card_transfer, client_freq_chat, 
               client_freq_card_recharge, client_freq_invest
              ]

In [28]:
#%%timeit
#client_freq_main_screen(data.multi_class_target)

In [45]:
%%time
# 5min 46s

client_freq = data_train.groupby('client_pin').multi_class_target.agg(cli_for_agg)
client_freq.to_csv(os.path.join(DATA_OWN, 'client_freq.csv'))
client_freq.head(10)

Wall time: 6min 1s


,client_freq_main_screen,client_freq_statement,client_freq_credit_info,client_freq_own_transfer,client_freq_mobile_recharge,client_freq_phone_money_transfer,client_freq_card2card_transfer,client_freq_chat,client_freq_card_recharge,client_freq_invest
client_pin,,,,,,,,,,
000033b6509acd1c8eb0d06ebd2e1de9,0.969466,0.015267,0.000000,0.0,0.015267,0.000000,0.000000,0.000000,0.000000,0.000000
0003653d4fa5e963ef379d65dd1d26ef,0.000000,1.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
0003a7947510434f8248b5b6a1eba0fa,0.636364,0.272727,0.000000,0.0,0.000000,0.000000,0.000000,0.090909,0.000000,0.000000
0005e7be2d89541ceea6b2f0bb4bc790,0.092308,0.000000,0.507692,0.0,0.000000,0.000000,0.000000,0.153846,0.246154,0.000000
00067f6a7575df6f067b3677b6ebfadc,0.564103,0.307692,0.076923,0.0,0.025641,0.000000,0.000000,0.025641,0.000000,0.000000
000967564d2bd81452b32fb6947421ee,0.692308,0.051282,0.051282,0.0,0.025641,0.025641,0.000000,0.153846,0.000000,0.000000
00096befff35b861d19c6de26db86fd6,0.689655,0.068966,0.068966,0.0,0.034483,0.068966,0.034483,0.000000,0.000000,0.034483
000a540f0ea28469451d4a7ad9e9436d,0.454545,0.151515,0.000000,0.0,0.181818,0.010101,0.191919,0.010101,0.000000,0.000000
000a90f770c38f235ff73e0f2aa7d197,0.542373,0.067797,0.000000,0.0,0.322034,0.050847,0.016949,0.000000,0.000000,0.000000


#### client frequency of target for day of week

In [47]:
%%time
# 32min 55s

client_freq_dow = data_train.groupby(['client_pin', 'dow']).multi_class_target.agg(cli_for_agg)
# after read_csv multilevel will disapper
client_freq_dow.to_csv(os.path.join(DATA_OWN, 'client_freq_dow.csv'))
client_freq_dow.head(10)

Wall time: 34min 52s


client_freq_main_screen  \
client_pin                       dow                            
000033b6509acd1c8eb0d06ebd2e1de9 0                   1.000000   
                                 1                   1.000000   
                                 2                   0.952381   
                                 3                   1.000000   
                                 4                   1.000000   
                                 5                   0.866667   
                                 6                   0.875000   
0003653d4fa5e963ef379d65dd1d26ef 0                   0.000000   
                                 4                   0.000000   
0003a7947510434f8248b5b6a1eba0fa 0                   0.666667   

                                      client_freq_statement  \
client_pin                       dow                          
000033b6509acd1c8eb0d06ebd2e1de9 0                 0.000000   
                                 1                 0.000000   
                                 2                 0.000000   
                                 3                 0.000000   
                                 4                 0.000000   
                                 5                 0.066667   
                                 6                 0.125000   
0003653d4fa5e963ef379d65dd1d26ef 0                 1.000000   
                                 4                 1.000000   
0003a7947510434f8248b5b6a1eba0fa 0                 0.333333   

                                      client_freq_credit_info  \
client_pin                       dow                            
000033b6509acd1c8eb0d06ebd2e1de9 0                        0.0   
                                 1                        0.0   
                                 2                        0.0   
                                 3                        0.0   
                                 4                        0.0   
                                 5                        0.0   
                                 6                        0.0   
0003653d4fa5e963ef379d65dd1d26ef 0                        0.0   
                                 4                        0.0   
0003a7947510434f8248b5b6a1eba0fa 0                        0.0   

                                      client_freq_own_transfer  \
client_pin                       dow                             
000033b6509acd1c8eb0d06ebd2e1de9 0                         0.0   
                                 1                         0.0   
                                 2                         0.0   
                                 3                         0.0   
                                 4                         0.0   
                                 5                         0.0   
                                 6                         0.0   
0003653d4fa5e963ef379d65dd1d26ef 0                         0.0   
                                 4                         0.0   
0003a7947510434f8248b5b6a1eba0fa 0                         0.0   

                                      client_freq_mobile_recharge  \
client_pin                       dow                                
000033b6509acd1c8eb0d06ebd2e1de9 0                       0.000000   
                                 1                       0.000000   
                                 2                       0.047619   
                                 3                       0.000000   
                                 4                       0.000000   
                                 5                       0.066667   
                                 6                       0.000000   
0003653d4fa5e963ef379d65dd1d26ef 0                       0.000000   
                                 4                       0.000000   
0003a7947510434f8248b5b6a1eba0fa 0                       0.000000   

                                      client_freq_phone_money_transfer  \
client_pin             

#### client frequency of target for times of day

In [34]:
%%time
# 19min 58s

client_freq_tod = data_train.groupby(['client_pin', 'tod']).multi_class_target.agg(cli_for_agg)
# after read_csv multilevel will disapper
client_freq_tod.to_csv(os.path.join(DATA_OWN, 'client_freq_tod.csv'))
client_freq_tod.head(10)

Wall time: 19min 58s


client_freq_main_screen  \
client_pin                       tod                                
000033b6509acd1c8eb0d06ebd2e1de9 day                     1.000000   
                                 evening                 0.955556   
                                 morning                 0.935484   
                                 night                   1.000000   
0003653d4fa5e963ef379d65dd1d26ef day                     0.000000   
0003a7947510434f8248b5b6a1eba0fa day                     0.500000   
                                 evening                 0.500000   
                                 morning                 1.000000   
                                 night                   0.666667   
0005e7be2d89541ceea6b2f0bb4bc790 day                     0.069767   

                                          client_freq_statement  \
client_pin                       tod                              
000033b6509acd1c8eb0d06ebd2e1de9 day                   0.000000   
                                 evening               0.044444   
                                 morning               0.000000   
                                 night                 0.000000   
0003653d4fa5e963ef379d65dd1d26ef day                   1.000000   
0003a7947510434f8248b5b6a1eba0fa day                   0.500000   
                                 evening               0.500000   
                                 morning               0.000000   
                                 night                 0.000000   
0005e7be2d89541ceea6b2f0bb4bc790 day                   0.000000   

                                          client_freq_credit_info  \
client_pin                       tod                                
000033b6509acd1c8eb0d06ebd2e1de9 day                     0.000000   
                                 evening                 0.000000   
                                 morning                 0.000000   
                                 night                   0.000000   
0003653d4fa5e963ef379d65dd1d26ef day                     0.000000   
0003a7947510434f8248b5b6a1eba0fa day                     0.000000   
                                 evening                 0.000000   
                                 morning                 0.000000   
                                 night                   0.000000   
0005e7be2d89541ceea6b2f0bb4bc790 day                     0.418605   

                                          client_freq_own_transfer  \
client_pin                       tod                                 
000033b6509acd1c8eb0d06ebd2e1de9 day                           0.0   
                                 evening                       0.0   
                                 morning                       0.0   
                                 night                         0.0   
0003653d4fa5e963ef379d65dd1d26ef day                           0.0   
0003a7947510434f8248b5b6a1eba0fa day                           0.0   
                                 evening                       0.0   
                                 morning                       0.0   
                                 night                         0.0   
0005e7be2d89541ceea6b2f0bb4bc790 day                           0.0   

                                          client_freq_mobile_recharge  \
client_pin                       tod                                    
000033b6509acd1c8eb0d06ebd2e1de9 day                         0.000000   
                                 evening                     0.000000   
                                 morning                     0.064516   
                                 night                       0.000000   
0003653d4fa5e963ef379d65dd1d26ef day                         0.000000   
0003a7947510434f8248b5b6a1eba0fa day                         0.000000   
                                 evening                     0.000000   
                                 morning                     0.000000   
        

## get difference from client freq target

In [48]:
%%time
# 11.3 s

#client_diff_freq_dom  = pd.DataFrame(index = client_freq_dom.index)
client_diff_freq_dow  = pd.DataFrame(index = client_freq_dow.index)
#client_diff_freq_hour = pd.DataFrame(index = client_freq_hour.index)
client_diff_freq_tod  = pd.DataFrame(index = client_freq_tod.index)

for el in client_freq.keys():
#    client_diff_freq_dom['diff_' + el]   = client_freq[el] - client_freq_dom[el]
    client_diff_freq_dow['diff_' + el]   = client_freq[el] - client_freq_dow[el]
#    client_diff_freq_hour['diff_' + el]  = client_freq[el] - client_freq_hour[el]
    client_diff_freq_tod['diff_' + el]   = client_freq[el] - client_freq_tod[el]
    
#client_diff_freq_dom.to_csv(os.path.join(DATA_OWN, 'client_diff_freq_dom.csv'))
client_diff_freq_dow.to_csv(os.path.join(DATA_OWN, 'client_diff_freq_dow.csv'))
#client_diff_freq_hour.to_csv(os.path.join(DATA_OWN, 'client_diff_freq_hour.csv'))
client_diff_freq_tod.to_csv(os.path.join(DATA_OWN, 'client_diff_freq_tod.csv'))

Wall time: 10.8 s


In [49]:
#del client_diff_freq_dom
#del client_diff_freq_hour

del client_freq
del client_freq_dow
del client_diff_freq_dow
del client_freq_tod
del client_diff_freq_tod

gc.collect()

30

In [50]:
del data_train
gc.collect()

22

when we get features below we should get them for test and train data    
for that we concat this dataframes

In [8]:
%%time
# 1min 7s

data_train = pd.read_csv(os.path.join(DATA, 'alfabattle2_abattle_train_target.csv'), parse_dates=['timestamp'])#.sort_values(by = ['timestamp'])
data_test = pd.read_csv(os.path.join(DATA, 'alfabattle2_prediction_session_timestamp.csv'), parse_dates=['timestamp'])#.sort_values(by = ['timestamp'])
data = pd.concat([data_train, data_test])
data.sort_values(by = ['timestamp'], inplace = True)

data.reset_index(inplace = True)
data.drop('index', axis = 1, inplace = True)
data.index.name = 'ind'

data['time_diff'] = data.groupby(['client_pin']).timestamp.diff()#.agg('size')
data['time_diff'] = data.time_diff.apply(lambda x: x.total_seconds() / 3600)
data['time_diff'].fillna(0, inplace = True)

#data.head()
data.tail()

Wall time: 1min 7s


,session_id,client_pin,timestamp,multi_class_target,time_diff
ind,,,,,
5144613,NaN,4e46e206d38dcb7af41d9ced20d50ac2,2020-09-30 23:41:47,NaN,55.986667
5144614,NaN,5b59bf5bd1e07775b1bc95dd7151d2cd,2020-09-30 23:43:25,NaN,37.122778
5144615,NaN,94bcc852dd7f806f6c733056d1b0be7b,2020-09-30 23:50:37,NaN,41.746389
5144616,NaN,2061f779928217eea2878fd3b605fafd,2020-09-30 23:53:03,NaN,6.094167
5144617,NaN,e2f0cede9e3bc3d6ed7c77c4e08af136,2020-09-30 23:58:32,NaN,2405.099167


## get last target before this session

In [9]:
def last_target(inp_ser):
    global dict_last_targets
    
    ret = dict_last_targets[inp_ser.client_pin.values[0]]
    dict_last_targets[inp_ser.client_pin.values[0]] = inp_ser.multi_class_target.values[0]
    
    return ret

In [60]:
%%time
# 5min 47s

dict_last_targets = {el: 'first_appear' for el in data.client_pin.unique()}
data_new = data.groupby(['client_pin', 'timestamp']).progress_apply(last_target)#.value_counts()
data_new = pd.DataFrame(data_new.reset_index())
data_new.columns = ['client_pin', 'timestamp', 'last_target_begore']
data_new.shape

100%|█████████████████████████████████████████████████████████████████████| 5144618/5144618 [05:39<00:00, 15160.09it/s]


Wall time: 5min 47s


(5144618, 3)

In [61]:
data_new.to_csv(os.path.join(DATA_OWN, 'last_target_begore.csv'))
pickle.dump(dict_last_targets, open(os.path.join(DATA_OWN, 'client_last_target.pkl'), 'wb'))

In [62]:
data_new.head()

,client_pin,timestamp,last_target_begore
0,000033b6509acd1c8eb0d06ebd2e1de9,2020-02-02 22:31:04,first_appear
1,000033b6509acd1c8eb0d06ebd2e1de9,2020-02-19 18:37:26,statement
2,000033b6509acd1c8eb0d06ebd2e1de9,2020-02-22 23:18:45,main_screen
3,000033b6509acd1c8eb0d06ebd2e1de9,2020-02-27 17:29:57,main_screen
4,000033b6509acd1c8eb0d06ebd2e1de9,2020-03-04 22:39:45,main_screen


In [63]:
data_new.last_target_begore.value_counts()

main_screen             2280763
statement                922569
credit_info              498698
own_transfer             290077
mobile_recharge          266485
phone_money_transfer     232911
card2card_transfer       193378
chat                     184775
card_recharge            138616
first_appear              79268
invest                    57078
Name: last_target_begore, dtype: int64

In [65]:
data_check = pd.merge(data, data_new,  how='left', left_on=['client_pin', 'timestamp'], right_on = ['client_pin','timestamp'])
print(data_check.isnull().values.any())
del data_check
gc.collect()

True


133

In [66]:
#data_check[data_check.client_pin == data_check.client_pin.unique()[682]]

In [67]:
del data_new
gc.collect()

44

# USED UNTIL THIS LINE

# time past after last known target for this user

In [19]:
target_dict = {'main_screen': 0, 
               'statement': 1,
               'credit_info': 2,
               'own_transfer': 3,
               'mobile_recharge': 4,
               'phone_money_transfer': 5,
               'card2card_transfer': 6,
               'chat': 7,
               'card_recharge': 8,
               'invest': 9
}

def target_time_diff(inp_data):

    #print(inp_data.shape)
    #print(inp_data.client_pin)
    #ret_series = pd.Series([[0]]*inp_data.shape[0]
    #                       , index = inp_data.index)
    ret_series = pd.Series(['']*inp_data.shape[0]
                           #, index = inp_data.index
                           , index = inp_data.timestamp
                           , dtype = 'object'
                          )
    last_known = [0] * len(target_dict)
    #last_known = np.array([0] * len(target_dict))
    data_len = (inp_data.shape[0] - 1)
    
    #print(ret_series.dtype)
    for idx, val in enumerate(inp_data.index):
        time  = inp_data['time_diff'][val]
        field = inp_data['multi_class_target'][val]
        
        last_known = [x+time for x in last_known]
        #ret_series.loc[val] = last_known
        #ret_series.iloc[idx] = np.array(last_known, dtype = object)
        ret_series.iloc[idx] = ' '.join([str(el) for el in last_known])
        #update last known time:
        if idx < data_len:
            last_known[target_dict[field]] = 0
            
    #print(ret_series.shape)
    
    return ret_series

In [20]:
%%time
# 34min 39s
# 37min 13s
#tt = first.apply(target_time_diff)

time_past_data = data.groupby('client_pin').progress_apply(target_time_diff)
time_past_data = time_past_data.reset_index()
time_past_data.columns = ['client_pin', 'timestamp', 'last_for_all_sessions']

100%|████████████████████████████████████████████████████████████████████████████| 79268/79268 [37:10<00:00, 35.54it/s]


Wall time: 37min 13s


In [25]:
%%time
# 9.73 s
# 7.72 s

#pickle.dump(time_past_data, open(os.path.join(DATA_OWN, 'time_past_no_processing.pkl'), 'wb'))
#time_past_data = pickle.load(open(os.path.join(DATA_OWN, 'time_past_no_processing.pkl'), 'rb'))   

Wall time: 9.15 s


In [27]:
%%time
1min 18s

time_past_data['last_for_all_sessions_split'] = time_past_data.last_for_all_sessions.apply(lambda x: x.split())

for el in target_dict.keys():
    #print(el)
    time_past_data['time_past_last_' + el] = time_past_data['last_for_all_sessions_split'].apply(lambda x: round(float(x[target_dict[el]]), 4))
    

Wall time: 1min 18s


In [30]:
time_past_data.drop('last_for_all_sessions', axis = 1, inplace = True)
time_past_data.drop('last_for_all_sessions_split', axis = 1, inplace = True)
time_past_data.head()

,client_pin,timestamp,time_past_last_main_screen,time_past_last_statement,time_past_last_credit_info,time_past_last_own_transfer,time_past_last_mobile_recharge,time_past_last_phone_money_transfer,time_past_last_card2card_transfer,time_past_last_chat,time_past_last_card_recharge,time_past_last_invest
0,000033b6509acd1c8eb0d06ebd2e1de9,2020-02-02 22:31:04,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
1,000033b6509acd1c8eb0d06ebd2e1de9,2020-02-19 18:37:26,404.1061,404.1061,404.1061,404.1061,404.1061,404.1061,404.1061,404.1061,404.1061,404.1061
2,000033b6509acd1c8eb0d06ebd2e1de9,2020-02-22 23:18:45,76.6886,480.7947,480.7947,480.7947,480.7947,480.7947,480.7947,480.7947,480.7947,480.7947
3,000033b6509acd1c8eb0d06ebd2e1de9,2020-02-27 17:29:57,114.1867,594.9814,594.9814,594.9814,594.9814,594.9814,594.9814,594.9814,594.9814,594.9814
4,000033b6509acd1c8eb0d06ebd2e1de9,2020-03-04 22:39:45,149.1633,744.1447,744.1447,744.1447,744.1447,744.1447,744.1447,744.1447,744.1447,744.1447


In [31]:
%%time
time_past_data.to_csv(os.path.join(DATA_OWN, 'time_past_last_target.csv'))
#time_past_data = pd.read_csv(os.path.join(DATA_OWN, 'time_past_last_target.csv'))

Wall time: 1min 6s


### Average time between targets

In [46]:
data = data.merge(time_past_data, how='left', on=['client_pin', 'timestamp'], validate='one_to_one')

In [33]:
target_dict.keys()

dict_keys(['main_screen', 'statement', 'credit_info', 'own_transfer', 'mobile_recharge', 'phone_money_transfer', 'card2card_transfer', 'chat', 'card_recharge', 'invest'])

In [48]:
data.keys()

Index(['session_id', 'client_pin', 'timestamp', 'multi_class_target',
       'time_diff', 'time_past_last_main_screen', 'time_past_last_statement',
       'time_past_last_credit_info', 'time_past_last_own_transfer',
       'time_past_last_mobile_recharge', 'time_past_last_phone_money_transfer',
       'time_past_last_card2card_transfer', 'time_past_last_chat',
       'time_past_last_card_recharge', 'time_past_last_invest'],
      dtype='object')

In [60]:
def target_hours_past_mean_v2(inp_data, targ):
    
    if targ in inp_data.multi_class_target.unique():
        return np.mean(inp_data['time_past_last_' + targ])
    else:
        return 0

In [62]:
#data.groupby('client_pin').apply(target_hours_past_mean_v2, targ = 'main_screen')

client_pin
000033b6509acd1c8eb0d06ebd2e1de9      39.617480
0003653d4fa5e963ef379d65dd1d26ef       0.000000
0003a7947510434f8248b5b6a1eba0fa     837.598917
0005e7be2d89541ceea6b2f0bb4bc790     432.199961
00067f6a7575df6f067b3677b6ebfadc     224.562793
                                       ...     
fffe62aa2ba7ac131c25ef6bde9fdb2f     303.824252
fffeb4fad3aa3603f65d7c847efbe93d    1200.025727
ffffa1546e530f390a721a535477fb26     196.844163
ffffdc3b688b6b4dbb1f468f12d4c9c5     213.103870
ffffe1150b0d2ca8005786cd35729226     165.185143
Length: 79268, dtype: float64

In [66]:
%%time

client_used_target_df = pd.DataFrame({'client_pin':data.client_pin.unique()},)#index = data.client_pin.unique())
for el in tqdm(target_dict.keys()):
    tmp_df = data.groupby('client_pin').apply(target_hours_past_mean_v2, targ = el)
    tmp_df.name = 'mean_' + el
    client_used_target_df = client_used_target_df.merge( tmp_df, how='left', on='client_pin', 
                                                        validate='one_to_one')

del tmp_df
gc.collect()

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [02:47<00:00, 16.71s/it]

Wall time: 2min 49s


In [68]:
client_used_target_df.head()

,client_pin,mean_main_screen,mean_statement,mean_credit_info,mean_own_transfer,mean_mobile_recharge,mean_phone_money_transfer,mean_card2card_transfer,mean_chat,mean_card_recharge,mean_invest
0,70783113e4f4117935d9f746237fce3e,346.811289,843.414292,0.000000,635.526722,2508.511989,1146.851525,0.000000,1393.139486,0.000000,0.000000
1,989f645a00e3e8a179036cf5fd5be29d,306.207385,260.987434,922.262046,1639.864346,1331.733406,1811.172441,1194.901347,610.430246,0.000000,0.000000
2,586fe605f61e81581efe72d9d3dc237d,1029.589168,935.371681,0.000000,0.000000,1374.284555,1378.438281,1478.086532,2516.315565,362.640345,2017.485923
3,c825cc9d1dfefe1fbc0fa1ead60fe2e1,816.068453,185.146866,0.000000,544.779027,106.022947,424.519171,802.808874,299.097450,1368.821794,0.000000
4,29928aba9d9bbf5e0377ddc6ec6f5294,707.222126,1479.208692,0.000000,29.489835,1401.347472,725.428054,1349.121669,2756.078173,0.000000,0.000000


In [69]:
%%time

client_used_target_df.to_csv(os.path.join(DATA_OWN, 'client_target_time_mean.csv'))
#client_used_target_df = pd.read_csv(os.path.join(DATA_OWN, 'client_target_time_mean.csv'))

Wall time: 1.05 s


### relations between target past time and target average time

In [89]:
data = data.merge(client_used_target_df, how='left', on='client_pin', validate='many_to_one')

In [ ]:
del client_used_target_df
gc.collect()

In [95]:
def set_relations(inp_data, targ):
    
    #global client_used_target_df
    #print(inp_data.shape)
    if inp_data['mean_' + targ] == 0:
        return 0
    else:
        return (inp_data['time_past_last_' + targ] / inp_data['mean_' + targ])

In [96]:
data.keys()

Index(['session_id', 'client_pin', 'timestamp', 'multi_class_target',
       'time_diff', 'time_past_last_main_screen', 'time_past_last_statement',
       'time_past_last_credit_info', 'time_past_last_own_transfer',
       'time_past_last_mobile_recharge', 'time_past_last_phone_money_transfer',
       'time_past_last_card2card_transfer', 'time_past_last_chat',
       'time_past_last_card_recharge', 'time_past_last_invest',
       'mean_main_screen', 'mean_statement', 'mean_credit_info',
       'mean_own_transfer', 'mean_mobile_recharge',
       'mean_phone_money_transfer', 'mean_card2card_transfer', 'mean_chat',
       'mean_card_recharge', 'mean_invest'],
      dtype='object')

In [100]:
%%time

for el in (target_dict.keys()):
    data['relations_time_past_targ_' + el] = data.progress_apply(lambda x: set_relations(x, targ = el), axis = 1 )

100%|█████████████████████████████████████████████████████████████████████| 5144618/5144618 [01:08<00:00, 75279.23it/s]


Wall time: 22min 38s


In [102]:
data.head()

,session_id,client_pin,timestamp,multi_class_target,time_diff,time_past_last_main_screen,time_past_last_statement,time_past_last_credit_info,time_past_last_own_transfer,time_past_last_mobile_recharge,...,relations_time_past_targ_main_screen,relations_time_past_targ_statement,relations_time_past_targ_credit_info,relations_time_past_targ_own_transfer,relations_time_past_targ_mobile_recharge,relations_time_past_targ_phone_money_transfer,relations_time_past_targ_card2card_transfer,relations_time_past_targ_chat,relations_time_past_targ_card_recharge,relations_time_past_targ_invest
0,da4ab6bb6795a4eb6458b88af7fb2367,70783113e4f4117935d9f746237fce3e,2020-01-01 00:00:00,mobile_recharge,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,f99d13f31091099672e51c1a02e4c567,989f645a00e3e8a179036cf5fd5be29d,2020-01-01 00:00:00,statement,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,69301aa932f73dec3355b803a63c39fe,586fe605f61e81581efe72d9d3dc237d,2020-01-01 00:00:01,statement,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,c6ae8307a0214b526a0e6fca3a9b80f5,c825cc9d1dfefe1fbc0fa1ead60fe2e1,2020-01-01 00:00:23,statement,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,3fd49317a617d521e0865496ec797550,29928aba9d9bbf5e0377ddc6ec6f5294,2020-01-01 00:00:35,mobile_recharge,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [108]:
data.keys()

Index(['session_id', 'client_pin', 'timestamp', 'multi_class_target',
       'time_diff', 'time_past_last_main_screen', 'time_past_last_statement',
       'time_past_last_credit_info', 'time_past_last_own_transfer',
       'time_past_last_mobile_recharge', 'time_past_last_phone_money_transfer',
       'time_past_last_card2card_transfer', 'time_past_last_chat',
       'time_past_last_card_recharge', 'time_past_last_invest',
       'mean_main_screen', 'mean_statement', 'mean_credit_info',
       'mean_own_transfer', 'mean_mobile_recharge',
       'mean_phone_money_transfer', 'mean_card2card_transfer', 'mean_chat',
       'mean_card_recharge', 'mean_invest',
       'relations_time_past_targ_main_screen',
       'relations_time_past_targ_statement',
       'relations_time_past_targ_credit_info',
       'relations_time_past_targ_own_transfer',
       'relations_time_past_targ_mobile_recharge',
       'relations_time_past_targ_phone_money_transfer',
       'relations_time_past_targ_card2card

In [109]:
for el in data.keys():
    if el.startswith('mean') or el.startswith('time_'):
        data.drop(el, inplace = True, axis = 1)

In [114]:
#data.drop('multi_class_target', inplace = True, axis = 1)
data.drop('session_id', inplace = True, axis = 1)

In [118]:
data.to_csv(os.path.join(DATA_OWN, 'relations_time_past_targ.csv'))

In [ ]:
del data
gc.collect()

# !!!!! HERE !!!!!

In [ ]:
#%%time
#with open(os.path.join(DATA_OWN, 'last_known2.pkl'), 'wb') as fd_last_known:
#    pickle.dump(tt, fd_last_known, protocol = 2)

In [ ]:
tt.index.name = ('client_pin', 'ind')

In [ ]:
%%time
#tt.to_csv(os.path.join(DATA_OWN, 'client_last_target.csv'))
dd = pd.read_csv(os.path.join(DATA_OWN, 'client_last_target.csv'))

In [ ]:
%%time
with open(os.path.join(DATA_OWN, 'last_known.pkl'), 'rb') as fd_last_known:
    tt2 = pickle.load(fd_last_known, protocol = 2)

In [ ]:
tt.index

In [ ]:
dd.columns = ['client_pin', 'ind', 'last_known']
dd.set_index('ind', inplace=True)

In [ ]:
dd.loc[4]

In [ ]:
data.loc[4]

In [ ]:
dd.head()

In [ ]:
for el in tqdm(target_dict.keys()):
    dd['last_known_' + el] = dd.last_known.apply(lambda x: x[target_dict[el]])

In [ ]:
target_dict.keys()

In [ ]:
dd

In [ ]:
dd.loc[0, 'last_known']

In [ ]:
from dask.distributed import Client
from dask import delayed

client = Client(n_workers=8)

In [ ]:
def prep_ds(inp_data):
    
    #global data
    #tmp_data = data.query('client_pin == @inp_client_pin')
    
    #return data.query('client_pin == @inp_client_pin').nunique()
    
    indexes = inp_data.index
    
    dict_freq = {'mobile_recharge': [0], 'statement': [0], 'phone_money_transfer': [0], 
             'chat': [0], 'invest': [0], 'main_screen': [0], 'own_transfer': [0], 
             'card_recharge': [0], 'credit_info': [0], 'card2card_transfer': [0]}
    
    tmp_value_counts = inp_data.multi_class_target.value_counts()
    sum_sess = inp_data.shape[0]
    for el in tmp_value_counts.keys():
        dict_freq[el] = [tmp_value_counts[el] / sum_sess]
    
    feature_dict = {}
    for idx, el in enumerate(indexes):
        feature_dict['last_before'] = 
        #'most_common',
        #'mc_last_24hours',
        #'mc_last_week',
        'days_since_last_days_since_last_mobile_recharge',
        'days_since_last_statement',
        'days_since_last_phone_money_transfer',
        'days_since_last_chat',
        'days_since_last_invest',
        'days_since_last_main_screen',
        'days_since_last_own_transfer',
        'days_since_last_card_recharge',
        'days_since_last_credit_info',
        'days_since_last_card2card_transfer'
    
    return inp_data.nunique()

In [ ]:
%%time
uniqq = []
for el in tqdm(data.client_pin.unique()[:100]):
    #uniqq.append(data.query('client_pin == @el').nunique())
    #ret = delayed(prep_ds)(el)
    ret = delayed(prep_ds)(data.query('client_pin == @el'))
    uniqq.append(ret)
    
ss = sum(uniqq)
answ = ss.compute()
#answ = sum(uniqq)
#ret.visalize()
print(answ)

In [ ]:
#ret.visualize()
client.close()

In [ ]:
data.client_pin.unique()[:5]